# Berechnung einer absoluten Positionierung mit Code-Messungen, Teil 2

In [2]:
# Requirements 
# pip install git+https://github.com/GNSSpy-Project/gnsspy
# pip install pyunpack
# pip install georinex

# Libs
import gnsspy as gp
import numpy as np
import georinex
import geopandas
import matplotlib.pyplot as plt
import math

# Params
np.set_printoptions(formatter={'float': '{: 0.5f}'.format})
plt.rcParams['figure.dpi'] = 300

In [7]:
%%capture
# Dataimport
station = gp.read_obsFile("./data/ONSA0320.11O")
ephemerides = georinex.load("./data/G3_11032.PRE")
clock = gp.read_clockFile("./data/cod16212.clk")

In [12]:
clock

,Epoch,DeltaTSV
G02,2011-02-01 00:00:00,0.000318
G03,2011-02-01 00:00:00,0.000670
G04,2011-02-01 00:00:00,0.000024
G05,2011-02-01 00:00:00,-0.000108
G06,2011-02-01 00:00:00,-0.000049
...,...,...
G28,2011-02-01 23:59:30,0.000018
G29,2011-02-01 23:59:30,0.000182
G30,2011-02-01 23:59:30,0.000318
G31,2011-02-01 23:59:30,0.000037
